## Chains ⛓️⛓️⛓️
Combining different LLM calls and action automatically

Ex: Summary #1, Summary #2, Summary #3 > Final Summary

Check out [this video](https://www.youtube.com/watch?v=f9_BWhCI4Zo&t=2s) explaining different summarization chain types

There are [many applications of chains](https://python.langchain.com/en/latest/modules/chains/how_to_guides.html) search to see which are best for your use case.

We'll cover two of them:

### 1. Simple Sequential Chains
Easy chains where you can use the output of an LLM as an input into another. Good for breaking up tasks (and keeping your LLM focused)

In [19]:
import os

from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

openai_api_key = os.environ["OPENAI_API_KEY"]

llm = OpenAI(temperature=1, openai_api_key=openai_api_key)

In [20]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = prompt_template | llm

In [21]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

In [27]:
chains = location_chain.pipe(meal_chain)

In [28]:
chains.invoke(input={"user_location": "rome"})

{'user_meal': "\nOne classic dish from Rome is cacio e pepe, a simple yet delicious pasta dish made with only a few ingredients. It consists of spaghetti or tonnarelli pasta tossed with a sauce made from grated pecorino romano cheese and black pepper. The end result is a creamy, peppery and cheesy pasta that is a staple in traditional Roman cuisine. It's often served as a first course or as a side dish for meat or fish. This dish has been enjoyed in Rome since ancient times, making it a true classic that represents the rich culinary history of the city.",
 'text': 'To make cacio e pepe at home, bring a pot of salted water to a boil and cook the spaghetti or tonnarelli pasta according to package instructions until al dente. In a separate pan, toast freshly cracked black pepper until fragrant. Then, add the cooked pasta to the pan with the pepper and a splash of cooking water. Slowly sprinkle in grated pecorino romano cheese, stirring constantly until it forms a creamy sauce. Serve hot a

In [30]:
# pipe operator is implemented by __or__ magic method in runnable class
chains = location_chain | meal_chain

In [32]:
result = chains.invoke(input={"user_location": "rome"})
result

In [34]:
result.get('user_meal')

'One classic dish from Rome is cacio e pepe, which translates to "cheese and pepper". This simple yet flavorful pasta dish consists of spaghetti or bucatini tossed with Pecorino Romano cheese and freshly ground black pepper. The cheese and pepper are combined with starchy pasta water to create a creamy and delectable sauce. This dish has been a staple of Roman cuisine for centuries and can be found in almost every traditional trattoria or restaurant in Rome. It is a must-try for anyone visiting the Eternal City.'

In [37]:
print(result.get('text'))

Recipe for Cacio e Pepe:

Ingredients:
- 1 pound spaghetti or bucatini pasta
- 1 cup grated Pecorino Romano cheese
- 2 tablespoons freshly ground black pepper
- Salt to taste

Instructions:
1. Bring a large pot of salted water to a boil. Add pasta and cook according to package instructions until al dente.

2. While the pasta is cooking, combine the grated Pecorino Romano cheese and black pepper in a small bowl.

3. Once the pasta is cooked, reserve 1 cup of the starchy pasta water before draining the pasta.

4. In a separate large pan, heat a tablespoon of olive oil over medium heat. Add the cooked pasta and a splash of the reserved pasta water to the pan and toss to coat.

5. Gradually add the cheese and pepper mixture to the pasta, stirring constantly until a creamy sauce forms. If the sauce is too thick, add more of the reserved pasta water.

6. Taste and add salt if necessary.

7. Serve hot, garnished with additional black pepper and grated Pecorino Romano cheese. Enjoy your homema

### branching chains
In LangChain v0.1 of langchain, we introduced the ability to create branching chains using the `BranchingChain` class. This allows you to create a chain that can take multiple paths based on the output of a previous chain.